In [1]:
from components.balloon import Balloon
from components.network import Network
from components.networkanalyzer import NetworkAnalyzer as na
from components.plothelper import PlotHelper as ph
from components.windmap import WindMap as wm
from components.sample_antennas import *

In [2]:
reconfigs = []
meaningful = []
for passes in range(0, 10):
    MyWindMap = wm(500, 500, 1)
    MyWindMap.create_windmap()
    MyWindMap.bound(4)
    
    MyPlotHelper = ph(MyWindMap)
    MyPlotHelper.populate(show_scale=False)
    # MyPlotHelper.plot(debug=False)
    
    B1 = Balloon(MyWindMap, "1", "red")
    B2 = Balloon(MyWindMap, "2", "blue")
    B3 = Balloon(MyWindMap, "3", "green")
    B4 = Balloon(MyWindMap, "4", "orange")
    B5 = Balloon(MyWindMap, "5", "purple")
    
    B1.set_start_pos(-20, -45)
    B2.set_start_pos(-20, -48)
    B3.set_start_pos(-21, -33)
    B4.set_start_pos(-20, -38)
    B5.set_start_pos(-20, -39)
    
    tickResponse_1 = 1
    tickResponse_2 = 1
    tickResponse_3 = 1
    tickResponse_4 = 1
    tickResponse_5 = 1

    B1.add_antenna(SaturnPatch)
    B2.add_antenna(SaturnPatch)
    B3.add_antenna(SaturnPatch)
    B4.add_antenna(SaturnPatch)
    B5.add_antenna(SaturnPatch)
    
    # MyPlotHelper.plot_balloons(B1, B2, B3, B4, B5)
    # MyPlotHelper.zoom_in(100, 100)
    
    NA = na(B1, B2, B3, B4, B5)
    # MyNetworkAnalyzer.show_pointing(100, MyFirstBalloon, MySecondBalloon, MyThirdBalloon, MyFourthBalloon)
    
    MyNetwork = Network(B1,
                        B2,
                        B3,
                        B4,
                        B5,
                        start=B1,
                        end=B5)
    
    while tickResponse_1:
        tickResponse_1 = B1.tick()
    while tickResponse_2:
        tickResponse_2 = B2.tick()
    while tickResponse_3:
        tickResponse_3 = B3.tick()
    while tickResponse_4:
        tickResponse_4 = B4.tick()
    while tickResponse_5:
        tickResponse_5 = B5.tick()
    
    for i in range(1, 8000):
        MyNetwork.recalculate(i, NA.rp)
        MyNetwork.transmit(i)
    
    print("\n== SIMULATION", passes, "COMPLETE ==")
    print("Reconfigurations:", len(MyNetwork.reconfigures))
    reconfigs.append(len(MyNetwork.reconfigures))
    print("Meaningful (above threshhold) reconfigs:", len(list(filter(lambda x: x['meaningful'] == True, MyNetwork.reconfigures))))
    print("Percentage meaningful:", (len(list(filter(lambda x: x['meaningful'] == True, MyNetwork.reconfigures)))/len(MyNetwork.reconfigures))*100, "\n")
    meaningful.append(len(list(filter(lambda x: x['meaningful'] == True, MyNetwork.reconfigures))))

print("== ALL SIMS COMPLETE ==")
print("Mean reconfigurations:", np.mean(reconfigs))
print("Mean meaningful reconfigurations:", np.mean(meaningful))
print("St.dev of reconfigurations:", np.std(reconfigs))
print("Variance of meaningful reconfigurations:", np.var(meaningful))

1  popped at  6539.5
1  hit the ground at time t= 8806.5
2  popped at  6582.0
2  hit the ground at time t= 8844.5
3  popped at  6445.5
3  hit the ground at time t= 8709.5
4  popped at  7138.5
4  hit the ground at time t= 9402.0
5  popped at  6583.5
5  hit the ground at time t= 8850.5

== SIMULATION 0 COMPLETE ==
Reconfigurations: 256
Meaningful (above threshhold) reconfigs: 84
Percentage meaningful: 32.8125 

1  popped at  6210.5
1  hit the ground at time t= 8474.5
2  popped at  6199.5
2  hit the ground at time t= 8465.5
3  popped at  7730.5
3  hit the ground at time t= 9994.5
4  popped at  6076.0
4  hit the ground at time t= 8343.5
5  popped at  7614.0
5  hit the ground at time t= 9880.5

== SIMULATION 1 COMPLETE ==
Reconfigurations: 269
Meaningful (above threshhold) reconfigs: 89
Percentage meaningful: 33.08550185873606 

1  popped at  7302.0
1  hit the ground at time t= 9563.0
2  popped at  7813.0
2  hit the ground at time t= 10077.0
3  popped at  6153.5
3  hit the ground at time t=

KeyboardInterrupt: 